In [ ]:
import os
import time
import argparse
import numpy as np
from visdom import Visdom
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from utils import *
import dataset

# Teacher models:
# VGG11/VGG13/VGG16/VGG19, GoogLeNet, AlxNet, ResNet18, ResNet34, 
# ResNet50, ResNet101, ResNet152, ResNeXt29_2x64d, ResNeXt29_4x64d, 
# ResNeXt29_8x64d, ResNeXt29_32x64d, PreActResNet18, PreActResNet34, 
# PreActResNet50, PreActResNet101, PreActResNet152, 
# DenseNet121, DenseNet161, DenseNet169, DenseNet201, 
# resnet20:ResNet20, ResNet32, ResNet44, ResNet56, ResNet110, ResNet1202
import models

# Student models:
# myNet, LeNet, FitNet


# main
def main():
    parser = argparse.ArgumentParser(description='cifar model with PyTorch')
    parser.add_argument('--dataset',
                    choices=['MNIST',
                             'CIFAR10',
                             'CIFAR100',
                             'tinyimagenet'
                            ],
                    default='CIFAR10')
    parser.add_argument('--model',
                        choices=['ResNet8',
                                 'ResNet15',
                                 'ResNet16',
                                 'ResNet20',
                                 'ResNet32',
                                 'ResNet44',
                                 'ResNet56',
                                 'ResNet110',
                                 'ResNet34',
                                 'ResNet50',
                                 'ResNet101',
                                 'DenseNet121',
                                 'VGG19'
                                ],
                        default='ResNet110',
                        help='name of net to train')
    parser.add_argument('--n_class', default=10, type=int, help='num of classes')
    parser.add_argument('--lr', default=0.001, type=float, help='learning rate')
    parser.add_argument('--epoch', default=50, type=int, help='number of epoch tp train for') 
    parser.add_argument('--trainBatchSize', default=128, type=int, help='training batch size')
    parser.add_argument('--testBatchSize', default=128, type=int, help='testing batch size')
    parser.add_argument('--device', default='cuda:0', type=str, help='device: cuda or cpu')
    parser.add_argument('--print_freq', '-p', default=20, type=int, metavar='N', help='print frequency (default: 20)')
    parser.add_argument('--save_every', dest='save_every', type=int, default=10, help='Saves checkpoints at every specified number of epochs')
    parser.add_argument('--save_dir', default='./checkpoint/', type=str, help='model and log saving dir')
    
    config_list = ['--dataset', 'CIFAR100', '--model', 'ResNet15', '--n_class', '100', '--lr', '0.1', '--epoch', '200', 
                   '--trainBatchSize', '128', '--testBatchSize', '128', '--device', 'cuda:0']
    args = parser.parse_args(config_list)
    args.save_dir += args.dataset + '/'
    os.makedirs(args.save_dir, exist_ok=True)
    
    solver = Solver(args)
    start_time = time.time()
    solver.run()
    end_time = time.time()
    print('time cost: {:.2f}min'.format((end_time - start_time)/60))


# Solver
class Solver(object):
    def __init__(self, config):
        self.model = None
        self.name = config.model
        self.dataset = config.dataset
        self.n_class = config.n_class
        self.lr = config.lr
        self.epochs = config.epoch
        self.train_batch_size = config.trainBatchSize
        self.test_batch_size = config.testBatchSize
        self.criterion = None
        self.optimizer = None
        self.scheduler = None
        self.device = config.device if torch.cuda.is_available() else 'cpu'
        self.train_loader = None
        self.test_loader = None
        self.print_freq = config.print_freq
        self.save_every = config.save_every
        self.save_dir = config.save_dir
        self.logfile = self.save_dir + self.name + '.log'
        if os.path.exists(self.logfile):
            os.remove(self.logfile)

        
    def print_model(self):
        print(self.model)
        
    def log_out(self, info):
        f = open(self.logfile, mode='a')
        f.write(str(info))
        f.write('\n')
        f.close()
        print(info)
        
    def load_data(self):
        normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        train_transform = transforms.Compose([
            transforms.RandomHorizontalFlip(),
            transforms.RandomCrop(32, 4),
            transforms.ToTensor(),
            normalize,
        ])
        test_transform = transforms.Compose([transforms.ToTensor(), normalize])
        if self.dataset == 'tinyimagenet':
            train_set = dataset.TinyImageNet(root='../data/tiny-imagenet-200', transform=train_transform)
            test_set = dataset.TinyImageNet(root='../data/tiny-imagenet-200', transform=test_transform)
        else:
            train_set = getattr(datasets, self.dataset)(root='../data', train=True, download=True, transform=train_transform)
            test_set = getattr(datasets, self.dataset)(root='../data', train=False, download=False, transform=test_transform)
        self.train_loader = DataLoader(train_set, batch_size=self.train_batch_size, shuffle=True, num_workers=4, pin_memory=True)
        self.test_loader = DataLoader(test_set, batch_size=self.test_batch_size, shuffle=False, num_workers=4, pin_memory=True)
        
    def load_model(self):
        self.model = getattr(models, self.name)(num_classes=self.n_class)
        self.print_model()
        self.model = self.model.to(self.device)

        self.optimizer_sgd = optim.SGD(self.model.parameters(), lr=self.lr, momentum=0.9, weight_decay=5e-4)
        self.lr_scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer_sgd, milestones=[100, 150])
        self.optimizer = optim.Adam(self.model.parameters(), lr=self.lr)
        self.scheduler = optim.lr_scheduler.MultiStepLR(self.optimizer, milestones=[75, 150], gamma=0.5)
        self.criterion = nn.CrossEntropyLoss().to(self.device)
    
    # train
    def train(self):
        print('Training:')
        # switch to train mode
        self.model.train()
        batch_time = AverageMeter()
        data_time = AverageMeter()
        losses = AverageMeter()
        top1 = AverageMeter()

        end = time.time()
        for i, (input, target) in enumerate(self.train_loader):

            # measure data loading time
            data_time.update(time.time() - end)

            input, target = input.to(self.device), target.to(self.device)

            # compute output
            self.optimizer_sgd.zero_grad()
            _,_,_,_,output = self.model(input)
            loss = self.criterion(output, target)

            # compute gradient and do SGD step
            loss.backward()
            self.optimizer_sgd.step()

            output = output.float()
            loss = loss.float()
            # measure accuracy and record loss
            train_acc = self.accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(train_acc, input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % self.print_freq == 0:
                self.log_out('[{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                          i, len(self.train_loader), batch_time=batch_time,
                          data_time=data_time, loss=losses, top1=top1))
        return losses.avg, train_acc.cpu().numpy()
    
    # test
    def test(self):
        print('Testing:')
        # switch to evaluate mode
        self.model.eval()
        batch_time = AverageMeter()
        losses = AverageMeter()
        top1 = AverageMeter()
    
        end = time.time()
        with torch.no_grad():
            for i, (input, target) in enumerate(self.test_loader):
                input, target = input.to(self.device), target.to(self.device)

                # compute output
                _,_,_,_,output = self.model(input)
                loss = self.criterion(output, target)

                output = output.float()
                loss = loss.float()

                # measure accuracy and record loss
                test_acc = self.accuracy(output, target)[0]
                losses.update(loss.item(), input.size(0))
                top1.update(test_acc, input.size(0))

                # measure elapsed time
                batch_time.update(time.time() - end)
                end = time.time()

                if i % self.print_freq == 0:
                    self.log_out('Test: [{0}/{1}]\t'
                          'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                          'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                          'Prec@1 {top1.val:.3f} ({top1.avg:.3f})'.format(
                              i, len(self.test_loader), batch_time=batch_time, loss=losses,
                              top1=top1))

        self.log_out(' * Prec@1 {top1.avg:.3f}'.format(top1=top1))

        return losses.avg, test_acc.cpu().numpy(), top1.avg.cpu().numpy()

    def save_model(self, epoch=None):
        if epoch != None:
            model_out_path = self.save_dir + self.name + '_' + str(epoch) +  '.pth'
        else:
            model_out_path = self.save_dir + self.name + '.pth'
        torch.save(self.model.state_dict(), model_out_path)
        self.log_out("* Checkpoint saved to {}".format(model_out_path))
    
    # compute accuracy
    def accuracy(self, output, target, topk=(1,)):
        """Computes the precision@k for the specified values of k"""
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res

    # run
    def run(self):
        start_time = time.time()
        self.log_out('*********{}********\n'.format(self.name))
        self.load_data()
        self.load_model() 
        accuracy = 0.
        
        vis = Visdom(env='training')
        loss_win = vis.line(
            X=np.array([0]),
            Y=np.array([0]),
            opts=dict(
                title=self.name + ' Loss',
                xlabel='epoch',
                xtickmin=0,
#                 xtickmax=1,
#                 xtickstep=5,
                ylabel='loss',
                ytickmin=0,
#                 ytickmax=1,
#                 ytickstep=0.5,
#                 markers=True,
#                 markersymbol='dot',
#                 markersize=5,
            ),
            name="loss"
        )
        
        acc_win = vis.line(
            X=np.column_stack((0, 0)),
            Y=np.column_stack((0, 0)),
            opts=dict(
                title=self.name + ' Acc',
                xlabel='epoch',
                xtickmin=0,
#                 xtickstep=5,
                ylabel='accuracy',
                ytickmin=0,
                ytickmax=100.0,
#                 ytickstep=5,
#                 markers=True,
#                 markersymbol='dot',
#                 markersize=5,
                legend=['train_acc', 'test_acc']
            ),
            name="acc"
        )
        
        best_acc = 0
        for epoch in range(1, self.epochs + 1):
            self.log_out("\n===> epoch: {}/{}".format(epoch, self.epochs))
            self.log_out('current lr {:.5e}'.format(self.optimizer_sgd.param_groups[0]['lr']))
            train_result = self.train()
            train_loss = train_result[0]
            train_acc = train_result[1]
            self.lr_scheduler.step()
            # visaulize loss
            vis.line(np.array([train_loss]), np.array([epoch]), loss_win, update="append")
#             self.log_out(train_result)
            test_result = self.test()
            test_acc = test_result[1]
            top1 = test_result[2]
#             best_acc = max(top1, best_acc)
            # save the best model.pth
            if top1 > best_acc:
                best_acc = top1
                if epoch > 150:
                    self.save_model()
            # visul accuracy
            vis.line(np.column_stack((train_acc, top1)), np.column_stack((epoch, epoch)), acc_win, update="append")
#             if epoch > 100 and epoch % self.save_every == 0:
#                 self.save_model(epoch)
        
        torch.cuda.empty_cache()
        self.log_out('\nBest ACC: {:.4f}%'.format(best_acc))
        self.log_out("--- {:.3f} mins ---".format((time.time() - start_time)/60))

# start 
if __name__ == '__main__':
    main()


*********ResNet15********

Files already downloaded and verified


/home/data/yaliu/jupyterbooks/multi-KD/models/student/resnet_s.py:13: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  init.kaiming_normal(m.weight)


ResNet(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(8, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(8, eps=1e-05, momentum=0.1, affine=True, track_ru


===> epoch: 1/200
current lr 1.00000e-01
Training:
[0/391]	Time 0.510 (0.510)	Data 0.387 (0.387)	Loss 5.9144 (5.9144)	Prec@1 0.000 (0.000)
[20/391]	Time 0.037 (0.081)	Data 0.000 (0.019)	Loss 4.5735 (4.8541)	Prec@1 3.125 (1.079)
[40/391]	Time 0.023 (0.068)	Data 0.000 (0.010)	Loss 4.5927 (4.7219)	Prec@1 1.562 (1.410)
[60/391]	Time 0.049 (0.062)	Data 0.000 (0.007)	Loss 4.5399 (4.6529)	Prec@1 2.344 (1.601)
[80/391]	Time 0.089 (0.062)	Data 0.000 (0.005)	Loss 4.4581 (4.6133)	Prec@1 1.562 (1.707)
[100/391]	Time 0.026 (0.060)	Data 0.001 (0.004)	Loss 4.4695 (4.5821)	Prec@1 2.344 (1.864)
[120/391]	Time 0.029 (0.060)	Data 0.000 (0.004)	Loss 4.4021 (4.5588)	Prec@1 3.125 (1.995)
[140/391]	Time 0.050 (0.060)	Data 0.000 (0.003)	Loss 4.4178 (4.5385)	Prec@1 3.125 (2.150)
[160/391]	Time 0.047 (0.060)	Data 0.001 (0.003)	Loss 4.4714 (4.5216)	Prec@1 3.125 (2.339)
[180/391]	Time 0.082 (0.059)	Data 0.000 (0.002)	Loss 4.3050 (4.5038)	Prec@1 4.688 (2.495)
[200/391]	Time 0.062 (0.059)	Data 0.000 (0.002)	Loss 4

[360/391]	Time 0.035 (0.059)	Data 0.000 (0.001)	Loss 3.1613 (3.1784)	Prec@1 21.094 (20.916)
[380/391]	Time 0.093 (0.059)	Data 0.000 (0.001)	Loss 3.0769 (3.1727)	Prec@1 19.531 (20.975)
Testing:
Test: [0/79]	Time 0.339 (0.339)	Loss 3.2194 (3.2194)	Prec@1 24.219 (24.219)
Test: [20/79]	Time 0.009 (0.034)	Loss 3.3545 (3.1930)	Prec@1 21.094 (21.540)
Test: [40/79]	Time 0.006 (0.024)	Loss 3.0847 (3.2131)	Prec@1 23.438 (20.884)
Test: [60/79]	Time 0.007 (0.019)	Loss 3.6187 (3.2256)	Prec@1 16.406 (20.838)
 * Prec@1 21.010

===> epoch: 5/200
current lr 1.00000e-01
Training:
[0/391]	Time 0.229 (0.229)	Data 0.201 (0.201)	Loss 3.1229 (3.1229)	Prec@1 23.438 (23.438)
[20/391]	Time 0.087 (0.058)	Data 0.000 (0.010)	Loss 3.1710 (3.0341)	Prec@1 18.750 (24.628)
[40/391]	Time 0.029 (0.058)	Data 0.000 (0.006)	Loss 2.8183 (3.0624)	Prec@1 28.125 (23.266)
[60/391]	Time 0.028 (0.059)	Data 0.000 (0.004)	Loss 3.1067 (3.0508)	Prec@1 23.438 (23.335)
[80/391]	Time 0.086 (0.060)	Data 0.000 (0.003)	Loss 2.9925 (3.0593)	

...

[200/391]	Time 0.029 (0.055)	Data 0.000 (0.001)	Loss 1.1529 (1.2575)	Prec@1 68.750 (63.316)
[220/391]	Time 0.039 (0.056)	Data 0.000 (0.001)	Loss 1.2378 (1.2592)	Prec@1 61.719 (63.267)
[240/391]	Time 0.024 (0.056)	Data 0.000 (0.001)	Loss 1.3171 (1.2629)	Prec@1 64.844 (63.197)
[260/391]	Time 0.077 (0.056)	Data 0.000 (0.001)	Loss 1.1847 (1.2679)	Prec@1 70.312 (63.054)
[280/391]	Time 0.031 (0.055)	Data 0.000 (0.001)	Loss 1.3626 (1.2702)	Prec@1 60.938 (63.034)
[300/391]	Time 0.039 (0.056)	Data 0.000 (0.001)	Loss 1.2727 (1.2710)	Prec@1 60.156 (63.037)
[320/391]	Time 0.022 (0.056)	Data 0.000 (0.001)	Loss 1.2354 (1.2736)	Prec@1 64.062 (62.953)
[340/391]	Time 0.032 (0.056)	Data 0.000 (0.001)	Loss 1.3207 (1.2748)	Prec@1 64.062 (62.910)
[360/391]	Time 0.031 (0.056)	Data 0.000 (0.001)	Loss 1.4585 (1.2785)	Prec@1 55.469 (62.851)
[380/391]	Time 0.073 (0.056)	Data 0.000 (0.001)	Loss 1.1720 (1.2795)	Prec@1 64.062 (62.840)
Testing:
Test: [0/79]	Time 0.239 (0.239)	Loss 1.4328 (1.4328)	Prec@1 57.812 (57.

[60/391]	Time 0.067 (0.055)	Data 0.000 (0.004)	Loss 1.3255 (1.2511)	Prec@1 65.625 (63.409)
[80/391]	Time 0.066 (0.055)	Data 0.000 (0.003)	Loss 1.3407 (1.2669)	Prec@1 57.812 (62.838)
[100/391]	Time 0.023 (0.055)	Data 0.000 (0.003)	Loss 1.1337 (1.2626)	Prec@1 67.188 (62.871)
[120/391]	Time 0.021 (0.054)	Data 0.000 (0.002)	Loss 1.1976 (1.2663)	Prec@1 63.281 (62.952)
[140/391]	Time 0.056 (0.055)	Data 0.000 (0.002)	Loss 1.4265 (1.2645)	Prec@1 55.469 (62.960)
[160/391]	Time 0.085 (0.055)	Data 0.000 (0.002)	Loss 1.3438 (1.2574)	Prec@1 57.031 (63.179)
[180/391]	Time 0.054 (0.055)	Data 0.000 (0.002)	Loss 1.3473 (1.2540)	Prec@1 60.156 (63.286)
[200/391]	Time 0.081 (0.055)	Data 0.000 (0.002)	Loss 1.4345 (1.2645)	Prec@1 57.031 (63.067)
[220/391]	Time 0.051 (0.055)	Data 0.000 (0.001)	Loss 1.3113 (1.2680)	Prec@1 66.406 (63.104)
[240/391]	Time 0.055 (0.054)	Data 0.000 (0.001)	Loss 1.1651 (1.2725)	Prec@1 64.062 (62.960)
[260/391]	Time 0.063 (0.053)	Data 0.000 (0.001)	Loss 1.4745 (1.2760)	Prec@1 57.812

Testing:
Test: [0/79]	Time 0.343 (0.343)	Loss 1.5218 (1.5218)	Prec@1 58.594 (58.594)
Test: [20/79]	Time 0.030 (0.033)	Loss 1.6938 (1.6035)	Prec@1 58.594 (56.027)
Test: [40/79]	Time 0.037 (0.026)	Loss 1.2137 (1.6031)	Prec@1 63.281 (55.774)
Test: [60/79]	Time 0.008 (0.021)	Loss 2.0599 (1.6216)	Prec@1 44.531 (55.149)
 * Prec@1 55.460

===> epoch: 131/200
current lr 1.00000e-02
Training:
[0/391]	Time 0.271 (0.271)	Data 0.211 (0.211)	Loss 1.2516 (1.2516)	Prec@1 61.719 (61.719)
[20/391]	Time 0.047 (0.053)	Data 0.000 (0.010)	Loss 1.2003 (1.2223)	Prec@1 66.406 (65.179)
[40/391]	Time 0.054 (0.048)	Data 0.000 (0.005)	Loss 1.2960 (1.2419)	Prec@1 58.594 (64.425)
[60/391]	Time 0.029 (0.045)	Data 0.000 (0.004)	Loss 1.3879 (1.2527)	Prec@1 60.156 (63.870)
[80/391]	Time 0.045 (0.045)	Data 0.000 (0.003)	Loss 1.0469 (1.2681)	Prec@1 71.875 (63.532)
[100/391]	Time 0.047 (0.046)	Data 0.000 (0.003)	Loss 1.0726 (1.2713)	Prec@1 65.625 (63.196)
[120/391]	Time 0.072 (0.048)	Data 0.001 (0.002)	Loss 1.1946 (1.2680

[260/391]	Time 0.081 (0.057)	Data 0.000 (0.002)	Loss 1.3020 (1.2660)	Prec@1 57.031 (63.347)
[280/391]	Time 0.035 (0.056)	Data 0.000 (0.002)	Loss 1.2482 (1.2675)	Prec@1 62.500 (63.306)
[300/391]	Time 0.037 (0.056)	Data 0.000 (0.002)	Loss 1.4266 (1.2718)	Prec@1 60.938 (63.196)
[320/391]	Time 0.025 (0.056)	Data 0.000 (0.002)	Loss 1.4697 (1.2699)	Prec@1 57.812 (63.181)
[340/391]	Time 0.071 (0.056)	Data 0.000 (0.001)	Loss 1.1151 (1.2734)	Prec@1 70.312 (63.132)
[360/391]	Time 0.043 (0.056)	Data 0.000 (0.001)	Loss 1.4445 (1.2744)	Prec@1 61.719 (63.084)
[380/391]	Time 0.109 (0.056)	Data 0.013 (0.001)	Loss 1.3439 (1.2755)	Prec@1 54.688 (63.017)
Testing:
Test: [0/79]	Time 0.276 (0.276)	Loss 1.4885 (1.4885)	Prec@1 56.250 (56.250)
Test: [20/79]	Time 0.007 (0.029)	Loss 1.5138 (1.5962)	Prec@1 64.062 (55.580)
Test: [40/79]	Time 0.035 (0.025)	Loss 1.1964 (1.5874)	Prec@1 63.281 (55.488)
Test: [60/79]	Time 0.017 (0.022)	Loss 1.9281 (1.5850)	Prec@1 46.875 (55.661)
 * Prec@1 55.650

===> epoch: 135/200
cu

[120/391]	Time 0.024 (0.057)	Data 0.000 (0.003)	Loss 1.2270 (1.2539)	Prec@1 66.406 (63.520)
[140/391]	Time 0.056 (0.055)	Data 0.000 (0.002)	Loss 1.0847 (1.2605)	Prec@1 66.406 (63.514)
[160/391]	Time 0.032 (0.053)	Data 0.000 (0.002)	Loss 1.0541 (1.2621)	Prec@1 70.312 (63.543)
[180/391]	Time 0.024 (0.051)	Data 0.000 (0.002)	Loss 1.2907 (1.2692)	Prec@1 61.719 (63.368)
